Partie IA

Imports

In [33]:
# Importation des bibliothèques nécessaires
import seaborn as sns  # Pour la visualisation statistique
import matplotlib.pyplot as plt  # Pour créer des graphiques
import pandas as pd  # Pour manipuler les données sous forme de DataFrame
from IPython.display import display, HTML  # Pour afficher du HTML dans un notebook Jupyter
import warnings  # Pour gérer les avertissements

# Importation des modules de Scikit-learn pour la transformation des colonnes, pipelines, modèles et évaluation
from sklearn.compose import ColumnTransformer  # Pour appliquer des transformations différentes sur les colonnes
from sklearn.pipeline import Pipeline  # Pour créer des pipelines de transformations et de modèles
from sklearn.preprocessing import StandardScaler  # Pour le prétraitement des données
from sklearn.model_selection import train_test_split  # Pour la division des données et la recherche des hyperparamètres
from sklearn.metrics import accuracy_score, classification_report  # Pour évaluer les performances du modèle
from catboost import CatBoostClassifier  # Classifieur CatBoost

# Ignorer certains avertissements
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')


In [34]:
df_tr = pd.read_csv(r'C:\Users\colin\Documents\ESEO\E5\docsPFE\creditcard_2023.csv')

In [35]:
# Séparation des données en caractéristiques (X) et cible (y)
X = df_tr.drop('Class', axis=1)
y = df_tr['Class']

1. df_tr.drop('Class', axis=1) :
        La méthode .drop() permet de supprimer une colonne spécifique du DataFrame.
        Ici, la colonne 'Class' (la variable cible) est supprimée du DataFrame df_tr pour créer l'ensemble des caractéristiques X.
        Le paramètre axis=1 indique que l'on supprime une colonne (et non une ligne).
        Résultat : X contient toutes les colonnes de df_tr à l'exception de la colonne 'Class'.

2. df_tr['Class'] :
        Ce code extrait uniquement la colonne 'Class' de df_tr, qui est assignée à la variable y, représentant la variable cible.
        Résultat : y contient les valeurs de la colonne 'Class', c'est-à-dire la variable que le modèle tentera de prédire.

En résumé :

> X : Contient toutes les colonnes de caractéristiques (features) à partir desquelles le modèle apprendra (toutes les colonnes sauf 'Class').<br>
> y : Contient la colonne 'Class', qui est la cible que l'on cherche à prédire (par exemple, pour un problème de classification binaire).

Ce processus est typique dans les tâches de machine learning, où on sépare les caractéristiques des étiquettes (target) avant d'entraîner un modèle.

In [36]:
# Définir les caractéristiques numériques (exclure les colonnes catégorielles)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

1. X.select_dtypes(include=['int64', 'float64']) :
        La méthode select_dtypes() permet de sélectionner uniquement les colonnes d'un certain type dans un DataFrame.
        Ici, le paramètre include=['int64', 'float64'] indique que seules les colonnes de type entier (int64) et flottant (float64) seront sélectionnées.
        Cela permet d'exclure automatiquement toutes les colonnes contenant des données catégorielles ou d'autres types non numériques, comme des chaînes de caractères (object) ou des booléens.

2. .columns.tolist() :
        La propriété .columns récupère les noms des colonnes sélectionnées dans le DataFrame.
        La méthode .tolist() convertit cette liste d'objets Index en une liste Python standard.

3. numeric_features :
        La variable numeric_features contiendra donc la liste des noms de toutes les colonnes numériques (entiers et flottants) dans X.

In [37]:
# Définir le pipeline de transformation pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

Pipeline :

> Un pipeline est une séquence d'étapes appliquées aux données, chacune représentant une transformation.

> Ici, il s'agit de définir un pipeline pour les caractéristiques numériques.

steps=[('scaler', StandardScaler())] :

> StandardScaler() est un transformateur qui normalise les données en les centrant sur la moyenne 0 et en les réduisant à une échelle unitaire (écart-type 1). Cela est souvent nécessaire avant de passer à des algorithmes d'apprentissage automatique, car certains modèles sont sensibles à l'échelle des données (par exemple, les modèles basés sur la distance comme la régression logistique, SVM, etc.).
> Le pipeline comporte une étape unique ici : la normalisation des colonnes numériques avec le StandardScaler.

In [38]:
# Définir la transformation par colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

ColumnTransformer :

> Il permet d'appliquer des transformations spécifiques à des sous-ensembles de colonnes d'un DataFrame. Cela est utile quand il y a un mélange de colonnes numériques et catégorielles nécessitant des prétraitements différents.

transformers=[...] :

> Ici, on spécifie que l'on veut appliquer des transformations aux colonnes numériques.

> ('num', numeric_transformer, numeric_features) :
>> 'num' : C'est une étiquette de nommage arbitraire pour cette transformation (utile pour identifier les étapes).
>> numeric_transformer : Le pipeline défini précédemment qui contient l'étape de mise à l'échelle des données numériques.
>> numeric_features : La liste des colonnes numériques sur lesquelles on applique cette transformation (sélectionnées plus tôt avec select_dtypes).

In [39]:
# Définir le modèle
model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', CatBoostClassifier(verbose=False))])

In [40]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_test_split :
    Cette fonction provient de la bibliothèque sklearn.model_selection et est utilisée pour diviser les données en deux ensembles distincts : un pour l'entraînement du modèle et un autre pour le test du modèle.
    Cela permet d'évaluer la performance du modèle sur des données qu'il n'a pas vues pendant l'entraînement, assurant ainsi que le modèle ne soit pas simplement surajusté (overfitted) aux données d'entraînement.

Paramètres de train_test_split :
    X : C'est le DataFrame contenant les caractéristiques (features) d'entrée.
    y : C'est la série contenant la variable cible (target) à prédire.
    test_size=0.2 :
        Ce paramètre spécifie la proportion de l'ensemble de données qui doit être utilisée comme ensemble de test.
        Ici, 20% des données seront utilisées pour le test, tandis que les 80% restants seront utilisés pour l'entraînement.
    random_state=42 :
        Ce paramètre fixe le graine aléatoire pour la génération de nombres aléatoires. Cela permet d'obtenir des résultats reproductibles.
        En utilisant une valeur fixe (comme 42 ici), chaque fois que vous exécuterez ce code, vous obtiendrez la même division entre les ensembles d'entraînement et de test.

Variables résultantes :
    X_train : Contient les caractéristiques des données d'entraînement.
    X_test : Contient les caractéristiques des données de test.
    y_train : Contient les étiquettes (cibles) des données d'entraînement.
    y_test : Contient les étiquettes (cibles) des données de test.

En résumé :

Ce code divise l'ensemble de données en deux parties : 80% pour l'entraînement et 20% pour le test.
Cela est essentiel pour évaluer correctement les performances d'un modèle de machine learning, en s'assurant que les évaluations de performance (précision, rappel, etc.) ne soient pas biaisées par l'utilisation des mêmes données pour l'entraînement et le test.

In [ ]:
# Entraîner le modèle
model.fit(X_train, y_train)



model.fit(...) :
    La méthode fit() est utilisée pour entraîner le modèle sur les données d'entraînement.
    Cela implique que le modèle va apprendre à partir des caractéristiques (X_train) et des étiquettes cibles (y_train) fournies.

Données d'entraînement :
    X_train : Contient les caractéristiques (features) d'entrée qui seront utilisées pour prédire les cibles. Chaque ligne représente un exemple d'entraînement, et chaque colonne représente une caractéristique.
    y_train : Contient les étiquettes (targets) correspondantes pour chaque exemple d'entraînement dans X_train. Cela signifie que pour chaque ensemble de caractéristiques dans X_train, il y a une étiquette correspondante dans y_train que le modèle doit apprendre à prédire.

Que se passe-t-il lors de l'exécution de fit() ?

Apprentissage : Le modèle (dans ce cas, un CatBoostClassifier) va parcourir les données d'entraînement, analyser les relations entre les caractéristiques et les cibles, et ajuster ses paramètres internes en conséquence.
Optimisation : Le modèle utilise des algorithmes d'optimisation pour minimiser la fonction de perte, qui quantifie à quel point les prédictions du modèle s'écartent des valeurs réelles de y_train.
Stockage des paramètres : Après l'entraînement, le modèle mémorise les informations nécessaires pour faire des prédictions sur de nouvelles données.

Importance de cette étape

Entraînement du modèle : C'est une étape essentielle dans le processus de machine learning, car c'est à ce moment-là que le modèle apprend à partir des données.
Préparation pour la prédiction : Une fois le modèle entraîné, il peut être utilisé pour faire des prédictions sur des données non vues (comme celles dans X_test) en utilisant la méthode predict().

En résumé :

Ce code entraîne le modèle défini précédemment en utilisant les données d'entraînement.
Une fois cette étape terminée, le modèle est prêt à faire des prédictions sur de nouvelles données en fonction des relations qu'il a apprises lors de l'entraînement.

In [25]:
# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)



model.predict(...) :
    La méthode predict() est utilisée pour générer des prédictions basées sur les caractéristiques de l'ensemble de test.
    Elle prend en entrée X_test, qui contient les caractéristiques des exemples de test que le modèle n'a jamais vus auparavant.

Données de test :
    X_test : Contient les caractéristiques des données qui seront utilisées pour évaluer la performance du modèle. Chaque ligne dans X_test représente un exemple sur lequel le modèle doit faire des prédictions.

Résultats des prédictions :
    y_pred : C'est un tableau ou une série qui contient les prédictions du modèle pour chaque exemple de test. Chaque valeur dans y_pred correspond à la classe ou la valeur prédite pour l'exemple de test correspondant dans X_test.

Que se passe-t-il lors de l'exécution de predict() ?

Génération des prédictions : Le modèle utilise les paramètres qu'il a appris lors de l'entraînement pour prédire la cible (classe ou valeur) de chaque exemple dans X_test.
Utilisation des caractéristiques : Pour chaque exemple dans X_test, le modèle applique les relations qu'il a apprises entre les caractéristiques et la variable cible.

Importance de cette étape

Évaluation des performances : Les prédictions effectuées sur l'ensemble de test (stockées dans y_pred) sont essentielles pour évaluer la performance du modèle en les comparant aux vraies valeurs de cibles correspondantes dans y_test.
Vérification de l'efficacité : Cela permet d'analyser la capacité du modèle à généraliser et à faire des prédictions sur des données qu'il n'a pas vues, ce qui est crucial pour comprendre sa performance dans des scénarios réels.

En résumé :

Ce code génère des prédictions pour l'ensemble de test en utilisant le modèle de machine learning déjà entraîné.
Les résultats sont stockés dans y_pred, ce qui permettra de les comparer aux vraies cibles pour évaluer la précision et l'efficacité du modèle.

In [26]:
# Calculer l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9996570705027874


accuracy_score(...) :
    accuracy_score est une fonction provenant de la bibliothèque sklearn.metrics. Elle est utilisée pour évaluer la performance du modèle en calculant l'exactitude des prédictions.
    La fonction prend deux arguments :
        y_test : Ce sont les vraies valeurs de la cible pour l'ensemble de test.
        y_pred : Ce sont les valeurs prédites par le modèle sur le même ensemble de test.

Calcul de l'exactitude :
    L'exactitude est définie comme le rapport du nombre de prédictions correctes au nombre total de prédictions effectuées. Elle peut être calculée comme suit :
Accuracy=Nombre de preˊdictions correctesNombre total de preˊdictions
Accuracy=Nombre total de preˊdictionsNombre de preˊdictions correctes​
    Une exactitude de 1 (ou 100%) signifie que toutes les prédictions étaient correctes, tandis qu'une exactitude de 0 signifie que le modèle n'a fait aucune prédiction correcte.

Stockage du résultat :
    accuracy : Cette variable stocke la valeur de l'exactitude calculée, qui peut ensuite être utilisée pour des analyses ou des comparaisons ultérieures.

Affichage du résultat :
    print("Accuracy:", accuracy) : Cela affiche la valeur de l'exactitude dans la console. Le texte "Accuracy:" est suivi de la valeur de l'exactitude calculée.

Importance de cette étape

Évaluation de la performance : L'exactitude est une métrique couramment utilisée pour évaluer les modèles de classification. Elle fournit une indication générale de la performance du modèle.
Comparaison de modèles : L'exactitude permet de comparer la performance de différents modèles ou configurations de modèles afin de sélectionner celui qui donne les meilleurs résultats.
Limites : Bien que l'exactitude soit une mesure utile, elle peut parfois être trompeuse, surtout dans des ensembles de données déséquilibrés (où certaines classes sont beaucoup plus fréquentes que d'autres). Dans ces cas, d'autres métriques comme la précision, le rappel ou le score F1 peuvent être plus informatives.

En résumé :

Ce code calcule l'exactitude du modèle sur l'ensemble de test en comparant les valeurs prédites aux vraies valeurs.
Le résultat est stocké dans la variable accuracy et est ensuite affiché à l'utilisateur, ce qui fournit une mesure directe de la performance du modèle

In [28]:
# Afficher le rapport de classification avec couleurs et en-tête
from colorama import Fore, Back, Style
print("\n" + Back.BLUE + Fore.WHITE + "Classification Report" + Style.RESET_ALL)
report = classification_report(y_test, y_pred, output_dict=True)
for key, value in report.items():
    if key in ['0', '1']:  
        color = Fore.GREEN if value['precision'] > 0.8 else Fore.RED
        print(f"Class {key}:")
        print(f"  Precision: {color}{value['precision']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['recall'] > 0.8 else Fore.RED
        print(f"  Recall: {color}{value['recall']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['f1-score'] > 0.8 else Fore.RED
        print(f"  F1-score: {color}{value['f1-score']:.2f}{Style.RESET_ALL}")
        print(f"  Support: {value['support']}")
    else:
        print(key + ":", value)


Classification Report
Class 0:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56750.0
Class 1:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56976.0
accuracy: 0.9996570705027874
macro avg: {'precision': 0.9996569545708367, 'recall': 0.9996571914660413, 'f1-score': 0.9996570692311153, 'support': 113726.0}
weighted avg: {'precision': 0.999657078158671, 'recall': 0.9996570705027874, 'f1-score': 0.9996570705434346, 'support': 113726.0}


Importation des modules :
    from colorama import Fore, Back, Style : La bibliothèque colorama est utilisée pour ajouter des couleurs et des styles au texte imprimé dans la console.
    Fore : Définit les couleurs de premier plan (texte).
    Back : Définit les couleurs d'arrière-plan.
    Style : Permet de réinitialiser les styles ou de définir des styles spéciaux.

Affichage de l'en-tête :
    print("\n" + Back.BLUE + Fore.WHITE + "Classification Report" + Style.RESET_ALL) :
        Affiche un en-tête avec un fond bleu et du texte blanc pour indiquer le début du rapport de classification.
        Style.RESET_ALL : Réinitialise le style à la valeur par défaut après l'impression de l'en-tête.

Génération du rapport de classification :
    report = classification_report(y_test, y_pred, output_dict=True) :
        Cette ligne génère un rapport de classification à l'aide de la fonction classification_report de sklearn.metrics.
        output_dict=True : Cela indique que le rapport doit être renvoyé sous forme de dictionnaire, ce qui facilite l'accès aux métriques pour chaque classe.

Affichage des résultats par classe :
    La boucle for key, value in report.items(): parcourt chaque élément du rapport.
    Conditions pour les classes :
        if key in ['0', '1']: : Cela vérifie si la clé représente une classe spécifique. Ici, les classes '0' et '1' sont traitées pour afficher des statistiques détaillées.

Mise en forme colorée :
    Pour chaque métrique (précision, rappel, score F1), la couleur du texte est définie en fonction de la valeur de la métrique :
        Précision : Si elle est supérieure à 0.8, la couleur est verte ; sinon, elle est rouge.
        Rappel : Même logique que pour la précision.
        F1-score : Même logique que pour la précision et le rappel.
    Chaque métrique est imprimée avec sa valeur formatée à deux décimales.

Affichage du support :
    print(f" Support: {value['support']}") : Affiche le nombre d'exemples pour chaque classe.

Autres clés :
    Si la clé ne correspond pas aux classes '0' ou '1', elle est simplement imprimée avec sa valeur.

Importance de cette étape

Évaluation détaillée : Le rapport de classification fournit des mesures clés pour chaque classe, ce qui permet d'évaluer la performance du modèle de manière détaillée.
Analyse visuelle : L'utilisation de couleurs rend les résultats plus faciles à interpréter et à comprendre rapidement.
Identification des classes problématiques : Les métriques comme la précision et le rappel permettent d'identifier les classes pour lesquelles le modèle pourrait avoir des difficultés, ce qui peut guider des ajustements futurs.

En résumé :

Ce code génère et affiche un rapport de classification détaillé pour le modèle.
Les métriques de performance sont présentées avec une mise en forme colorée pour faciliter l'analyse, permettant une évaluation rapide et efficace de la performance du modèle sur chaque classe.